# Основы обучения с учителем (supervised learning)

*Цель занятия* - рассмотреть на практике алгоритмы обучения с учителем. 
По ходу занятия полезно иметь под рукой документацию к библиотеке scikit-learn (http://scikit-learn.org)

Подключим необходимые нам пакеты:

In [ ]:
%matplotlib inline
# numpy - превращает питон в матлаб и крайне шустро перемножает матрицы
import numpy as np  
import matplotlib.pyplot as plt

# настроки графики
import seaborn; seaborn.set()

## Классификация и регрессия

Напомню вкратце - *обучение с учителем* подразумевает *наличие "ответов"* для объектов тренировочной выборки. 

В случае регрессии ответы - действительные числа, для классификации - дискретные классы.


### Регрессия

Для начала решим простую задачу линейной регрессии

In [ ]:
import numpy as np
# random.seed - просто для повторяемости эксперимента
np.random.seed(0)

# генерируем почти случайные данные
X = np.random.random(size=(20, 1))
y = 3 * X.squeeze() + 2 + np.random.randn(20)

plt.plot(X.squeeze(), y, 'o');

Решим задачу определения прямой методом наименьших квадратов. Другими словами, используем модель линейной регрессии:

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)

# рисуем данные и 
X_fit = np.linspace(0, 1, 100)[:, np.newaxis]
y_fit = model.predict(X_fit)

plt.plot(X.squeeze(), y, 'o')
plt.plot(X_fit.squeeze(), y_fit);

## ЗАДАНИЕ
Вспомните, на лекции мы рассматривали вариант линейной регрессии на нелинейно преобразованных признаках (полином или что душе угодно).

Нижеследующий код добавит в матрицу объект-признак два дополнительных преобразования. 

```
X = np.hstack(( X, X**2, np.sin(X) ))
X_fit = np.hstack(( X_fit, X_fit**2, np.sin(X_fit) )))
```

**Задание**: 
1. Скопировать код с регрессией
2. Добавить признаков. Что стало с регрессией?
3. Добавьте в конец кода строчку:

``` 
model.coef_ # веса ваших признаков 
```


### Регрессия с использованием решающего леса

Предыдущий пример можно решить с помощью другого классификатора.

**Еще задание**
Посмотрите http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
Попробуйте немного поиграть с параметрами n_estimators=10 (**не больше 200 !!!**) , min_samples_split, min_samples_leaf

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, y)

X_fit = np.linspace(0, 1, 100)[:, np.newaxis]
y_fit = model.predict(X_fit)

plt.plot(X.squeeze(), y, 'o')
plt.plot(X_fit.squeeze(), y_fit);

## Классификация. Простой пример

Рассмотрим простейший пример классификации на примере стандартного дата сета с цветками Ириса

<img src="images/iris_petal_sepal.png">

In [ ]:
# Загрузим дата сет
from sklearn.datasets import load_iris
iris = load_iris()

print(iris.DESCR)
print(iris.target_names) # названия переменных
print(iris.data)         # матрица объект-признак
print(iris.target)       # целевые переменные

Начнем с подготовки данных. Выделим 40% на тестирование и 60% на тренировку

In [ ]:
X = iris.data
y = iris.target

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=4)

Обучим простейший классификатор и изучим его работу

In [ ]:
from sklearn import tree
from sklearn import metrics
    
model = tree.DecisionTreeClassifier(min_samples_leaf=30)
model.fit(X_train, y_train)

# предсказание
y_pred = model.predict(X_test)

# вычислим точность модели по результатам классификации
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

Довольно хитрым способом нарисуем дерево:

In [ ]:
from IPython.display import Image 
from sklearn.externals.six import StringIO
import pydot

dotfile = StringIO()
tree.export_graphviz(model, out_file = dotfile, feature_names=iris.feature_names,  
                         class_names=iris.target_names,filled=True, rounded=True,  
                         special_characters=True)
Image(pydot.graph_from_dot_data(dotfile.getvalue())[0].create_png())

## ЗАДАНИЕ

Измените параметр min_samples_leaf. Как меняется точность и структура дерева?

# О выборе классификатора

In [ ]:
from IPython.display import Image
Image("http://scikit-learn.org/dev/_static/ml_map.png")

# Более реальный пример: распознавание изображений

MNIST digits - база ч/б картинок 8x8 пикселей.

До недавнего времени являлся стандартным дата сетом для оценки точности классификаторов. 

### Загрузка дата сета MNIST

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

digits = datasets.load_digits()
digits.images.shape

Нарисуем несколько изображений из MNIST:

In [ ]:
fig, axes = plt.subplots(10, 10, figsize=(8, 8))
fig.subplots_adjust(hspace=0.1, wspace=0.1)

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='binary', interpolation='nearest')
    ax.text(0.05, 0.05, str(digits.target[i]),
            transform=ax.transAxes, color='green')
    ax.set_xticks([])
    ax.set_yticks([])

Так выглядит матрица пикселей для одной картинки 
(меняйте индекс у images чтобы просмотреть другие изображения):

In [ ]:
print(digits.images.shape)
print(digits.images[0])

In [ ]:
# Для упрощения работы используется вектор из 64 значений вместо матрицы 8x8
print(digits.data.shape)
print(digits.data[0])

In [ ]:
# Ответы в данном случае - цифры
print(digits.target)

Таким образом, имеем 1797 объектов, 10 видов ответов и размерность вектора признаков 64.

### Классификация картинок

Начнем с подготовки данных. Разобъем объекты на тренировочные и тестовые

In [ ]:
from sklearn.cross_validation import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=2)
print(Xtrain.shape, Xtest.shape)

Начнем с логистической регрессии:

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2')

clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)

Такая получается точность:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, ypred)

Построим *матрицу перепутывания* (*confusion matrix*)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest, ypred))

Или в цвете (часто используется при публикации статей):

In [ ]:
plt.imshow(np.log(confusion_matrix(ytest, ypred)),
           cmap='Blues', interpolation='nearest')
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted');

Можно так же нарисовать, где и какие ошибки сделала наша система:

In [ ]:
fig, axes = plt.subplots(10, 10, figsize=(8, 8))
fig.subplots_adjust(hspace=0.1, wspace=0.1)

for i, ax in enumerate(axes.flat):
    ax.imshow(Xtest[i].reshape(8, 8), cmap='binary')
    ax.text(0.05, 0.05, str(ypred[i]),
            transform=ax.transAxes,
            color='green' if (ytest[i] == ypred[i]) else 'red')
    ax.set_xticks([])
    ax.set_yticks([])

# ЗАДАНИЕ

Попробуйте другие классификаторы из http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

Например
* SVC (Support Vector Classifier) с параметром kernel='poly' дает больше 0.97
* RandomForestClassifier на 100 деревьях дает 0.955

 

